In [1]:
import pandas as pd
import os
import numpy as np
import warnings
import seaborn as sns
import re
from tqdm import tqdm
warnings.filterwarnings('ignore')

# 학습 데이터 불러오기

데이터를 읽을 때 단순 pd.read_csv로 읽으면 error_bad_lines 관련 문제가 생겨서 pd.read_csv 대신 아래 셀과 같이 긴 작업을 거쳤습니다.

In [2]:
## 아래 모든 코드는 pd.read_csv의 대용입니다.

import csv
f = open('train.csv', 'r', encoding='utf-8')
rdr = csv.reader(f)
rr=0
lines=[]
for line in tqdm(rdr):
    if rr<=1000000000000000000000000000000000000000000:
        if len(line)==25:
            line_final=line[:9]+[line[9]+line[10]]+line[11:]
        elif len(line)==24:
            line_final=line
        rr+=1
        lines.append(line_final)
    else:
        break

columns = lines[0]
df = pd.DataFrame(lines[1:],columns=columns)
df = df.loc[(df['ord_dt']>='2020-08-16')&(df['ord_dt']<='2020-09-15')].reset_index().drop(['index'],axis=1)

10808186it [02:41, 67084.46it/s]


# val 데이터 변환

In [3]:
val = pd.read_csv("validation.csv")
val_date = val.groupby(['shop_no','ord_dt'])['abuse_yn'].mean().reset_index()

# train 데이터 만들기

In [5]:
df['ord_dts'] = pd.to_datetime(df['ord_dt']) # df에서 ord_dt 컬럼을 datetime 타입으로 바꾸어서 ord_dts 컬럼에 저장합니다.

In [6]:
mem_shop = df.groupby(['mem_no','shop_no']).size().reset_index().rename(columns={0:'mem_duplicate'})

df = pd.merge(df, mem_shop,on=['mem_no','shop_no'],how='left') # 각 상점별, 유저별로 빈도수를 비교합니다.

In [7]:
# 주문한 시간과 리뷰를 남긴 시간의 차이를 계산합니다.
df['diff_time'] = (pd.to_datetime(np.where(df['review_created_tm']=='\\N',np.nan,df['review_created_tm'])) - pd.to_datetime(df['ord_tm'])).dt.total_seconds()

In [8]:
numeric_columns=['mem_duplicate', 'diff_time','item_quantity','cpn_use_cnt','ord_price','review_yn','rating','image_review_yn']
sparse_cate_columns = ['ci_seq', 'mem_no', 'dvc_id','rgn1_cd', 'rgn2_cd', 'rgn3_cd']

In [9]:
# 수치형 변수여야 하는 컬럼을 float로 바꿉니다.

for col in numeric_columns:
    df[col]=df[col].astype(float)

In [10]:
# 상점, 날짜별 주문수를 추출합니다.
size = df.groupby(['shop_no','ord_dt']).size().reset_index().rename(columns={0:'size'})

In [11]:
# 상점, 날짜별로 sparse_cate_columns의 nunique를 추출합니다.

for col in sparse_cate_columns:
    size = pd.merge(size,df.groupby(['shop_no','ord_dt'])[col].nunique().reset_index(),on=['shop_no','ord_dt'],how='left')
    size = size.rename(columns={col:col+'_nunique'})

In [12]:
# 상점, 날짜별로 numeric_columns의 평균과 합을 추출합니다.

for col in numeric_columns:
    size = pd.merge(size,df.groupby(['shop_no','ord_dt'])[col].mean().reset_index(),on=['shop_no','ord_dt'],how='left')
    size = size.rename(columns={col:col+'_mean'})

    size = pd.merge(size,df.groupby(['shop_no','ord_dt'])[col].sum().reset_index(),on=['shop_no','ord_dt'],how='left')
    size = size.rename(columns={col:col+'_sum'})

In [13]:
# 답지와 merge합니다
size = pd.merge(size,val_date[['shop_no','ord_dt','abuse_yn']],on=['shop_no','ord_dt'],how='left')

In [14]:
# nunique를 size로 나누어 ratio 변수를 생성한다.
size['mem_no_ratio'] = size['mem_no_nunique']/size['size']
size['ci_seq_ratio'] = size['ci_seq_nunique']/size['size']

# 학습 데이터, 정답 데이터

In [15]:
full_data = size

In [16]:
test_data = full_data.loc[full_data['ord_dt']>'2020-09-10'].reset_index().drop(['index'],axis=1)
train_data = full_data.loc[full_data['ord_dt']<='2020-09-10'].reset_index().drop(['index'],axis=1)

In [17]:
train_data.to_csv("Final_data/train_data.csv",index=False)
test_data.to_csv("Final_data/test_data.csv",index=False)